In [1]:
import re
import src.utils as utils
import numpy as np
import tensorflow as tf
import src.benchmark as benchmark
from tqdm.auto import tqdm

In [2]:
# import src.generate_vocabulary as gv
#
# gv.generate_vocabulary(['.\\data\\data'], ['py'], 25000)

In [3]:
# with open('.\\data\\python100k_train.txt') as f:
#     data_dirs = f.readlines()
#
#
# data_dirs = [re.sub(r'/', r'\\', ('.\\data\\'+d)[:-1]) for d in data_dirs]
# data_dirs = np.random.choice(data_dirs, size=50000, replace=False)
#
# tokenized_files = utils.tokenize_data(data_dirs)

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from src.model import Model

vocab_size = 20001
embedding_dim = 64
rnn_units = 512
batch_size = 128
win_size = 10
model = Model(vocab_size, embedding_dim, rnn_units, batch_size, win_size, '.\\checkpoints\\model.h5')
model.prepare_predictions('.\\vocabulary.voc', '.\\checkpoints\\model.h5')
# print(model.get_prediction('', 100))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 64)             1280064   
_________________________________________________________________
conv1d (Conv1D)              (1, None, 64)             4160      
_________________________________________________________________
lstm (LSTM)                  (1, None, 512)            1181696   
_________________________________________________________________
dense (Dense)                (1, None, 20001)          10260513  
Total params: 12,726,433
Trainable params: 12,726,433
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
with open('.\\data\\my_eval.txt') as f:
    data_dirs = f.readlines()

np.random.seed(43)
data_dirs = [re.sub(r'/', r'\\', ('.\\data\\'+d)[:-1]) for d in data_dirs]
data_dirs = np.random.choice(data_dirs, size=9, replace=False)

proper_predictions, all_predictions = 0,0

In [6]:
progress = tqdm(data_dirs, position=0)

for file in progress:
    result = benchmark.single_benchmark(model, file, html=True)
    proper_predictions += result[0]
    all_predictions += result[1]

    if all_predictions != 0:
        progress.set_postfix({'accuracy' : proper_predictions/all_predictions})



